In [ ]:
from ultralytics import YOLO
import os, csv, cv2

In [11]:
person_model = YOLO("weight/yolov8s.pt")
face_model = YOLO("weight/yolov8n-face-lindevs.pt")
head_model = YOLO("weight/nano.pt")

In [ ]:
INPUT_VIDEO = "input_video.mp4"
CONFIDENCE = 0.35
IMGSZ = 960
SAVE_DIR = "out/"

In [15]:
OUT = f"{SAVE_DIR}detection.mp4"
CSV = f"{SAVE_DIR}count.csv"
cap = cv2.VideoCapture(INPUT_VIDEO)
w, h = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS) or 30
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUT, fourcc, fps, (w,h))

In [14]:
def to_list(result):
    return result.boxes.xyxy.cpu().numpy().astype(int).tolist(), result.boxes.conf.cpu().numpy().tolist()

In [19]:
with open(CSV, "w", newline = "") as f:
    wr = csv.writer(f)
    wr.writerow(["frame", "n_person", "n_face", "n_head"])
    frame = 0
    while True:
        ok, img = cap.read()
        if not ok: break
        frame += 1
        r_p = person_model.predict(img, classes = [0], conf = CONFIDENCE, imgsz = IMGSZ, verbose = False)[0]
        r_h = head_model.predict(img, conf = CONFIDENCE, imgsz = IMGSZ, verbose = False)[0]
        r_f = face_model.predict(img, conf = CONFIDENCE, imgsz = IMGSZ, verbose = False)[0]

        P, Pc = to_list(r_p)
        F, Fc = to_list(r_f)
        H, Hc = to_list(r_h)
        
        for x1, y1, x2, y2 in P: cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        for x1, y1, x2, y2 in F: cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)
        for x1, y1, x2, y2 in H: cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,255), 2)
        
        wr.writerow([frame, len(P), len(F), len(H)])
        writer.write(img)
        
cap.release(); writer.release()
print("Saved:", OUT, "CSV:", CSV)
        

Saved: outdetection.mp4 CSV: outcount.csv
